In [5]:
import os
import pandas as pd
import numpy as np
import json

offical_names = ['MinerU-0.9.3', 'Marker-0.2.17', 'Marker-1.2.3', 'Mathpix', 'docling', 'GOT-OCR', 'Nougat', 'GPT4o', 'Qwen2-VL-72B', 'InternVL2-Llama3-76B']
ocr_types = ['mineru1110_docyolo', 'marker', 'marker0106', 'mathpix1108', 'docling', 'GOT', 'nougat', 'gpt4o_long_prompt', 'QwenVL_long_prompt', 'internvl_long_prompt']
# offical_names = ocr_types

result_folder = './result'

# match_name = 'no_split'
match_name = 'quick_match'

In [6]:
# overall result

dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    save_dict = {}
    en_overall = []
    ch_overall = []
    for category_type, metric in [("text_block", "Edit_dist"), ("display_formula", "Edit_dist"), ("display_formula", "CDM"), ("table", "TEDS"), ("table", "Edit_dist"), ("reading_order", "Edit_dist")]:
        if metric == 'CDM':
            save_dict[category_type+'_'+metric+'_EN'] = '-'
            save_dict[category_type+'_'+metric+'_CH'] = '-'
        elif metric == "TEDS":
            save_dict[category_type+'_'+metric+'_EN'] = result[category_type]["page"][metric]["language: english"] * 100
            save_dict[category_type+'_'+metric+'_CH'] = result[category_type]["page"][metric]["language: simplified_chinese"] * 100
            # save_dict[category_type+'_'+metric+'_EN'] = '-'
            # save_dict[category_type+'_'+metric+'_CH'] = '-'
        else:
            save_dict[category_type+'_'+metric+'_EN'] = result[category_type]["page"][metric].get("language: english", np.NaN)
            save_dict[category_type+'_'+metric+'_CH'] = result[category_type]["page"][metric].get("language: simplified_chinese",np.NaN)
        if metric == "Edit_dist":
            en_overall.append(result[category_type]["page"][metric].get("language: english", np.NaN))
            ch_overall.append(result[category_type]["page"][metric].get("language: simplified_chinese",np.NaN))
    
    save_dict['overall_EN'] = sum(en_overall) / len(en_overall)
    save_dict['overall_CH'] = sum(ch_overall) / len(ch_overall)
    dict_list.append(save_dict)
    
df = pd.DataFrame(dict_list, index=offical_names).round(3)
# df.to_csv('./overall.csv')

df

,text_block_Edit_dist_EN,text_block_Edit_dist_CH,display_formula_Edit_dist_EN,display_formula_Edit_dist_CH,display_formula_CDM_EN,display_formula_CDM_CH,table_TEDS_EN,table_TEDS_CH,table_Edit_dist_EN,table_Edit_dist_CH,reading_order_Edit_dist_EN,reading_order_Edit_dist_CH,overall_EN,overall_CH
MinerU-0.9.3,0.061,0.211,0.278,0.577,-,-,78.622,62.145,0.180,0.344,0.079,0.288,0.150,0.355
Marker-0.2.17,0.141,0.303,0.667,0.868,-,-,53.470,44.892,0.678,0.714,0.138,0.306,0.406,0.548
Marker-1.2.3,0.080,0.315,0.530,0.883,-,-,67.564,49.207,0.619,0.685,0.114,0.340,0.336,0.556
Mathpix,0.101,0.358,0.306,0.454,-,-,77.048,67.148,0.243,0.320,0.105,0.275,0.189,0.352
docling,0.416,0.987,0.999,1.000,-,-,61.277,24.972,0.627,0.810,0.313,0.837,0.589,0.909
GOT-OCR,0.191,0.315,0.360,0.528,-,-,53.193,47.201,0.459,0.520,0.143,0.280,0.289,0.411
Nougat,0.367,0.998,0.488,0.941,-,-,39.902,0.000,0.572,1.000,0.384,0.954,0.453,0.973
GPT4o,0.146,0.409,0.425,0.606,-,-,71.977,62.870,0.234,0.329,0.128,0.251,0.233,0.399
Qwen2-VL-72B,0.253,0.251,0.468,0.572,-,-,59.503,66.413,0.551,0.518,0.254,0.223,0.381,0.391
InternVL2-Llama3-76B,0.353,0.290,0.543,0.701,-,-,63.004,60.243,0.547,0.555,0.317,0.228,0.440,0.443


In [7]:
# PDF type
dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["page"]["Edit_dist"])
df2 = pd.DataFrame(dict_list, index=offical_names)

reordered_df2 = df2.round(3)

selected_columns = reordered_df2[["data_source: book", "data_source: PPT2PDF", "data_source: research_report", "data_source: colorful_textbook", "data_source: exam_paper", "data_source: magazine", "data_source: academic_literature", "data_source: note", "data_source: newspaper"]]
# calculate mean
selected_columns['mean'] = selected_columns.mean(axis=1).round(3)
# selected_columns['variance'] = selected_columns.var(axis=1)
# selected_columns.to_csv('./data_source.csv')

selected_columns

/mnt/petrelfs/ouyanglinke/.local/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,data_source: book,data_source: PPT2PDF,data_source: research_report,data_source: colorful_textbook,data_source: exam_paper,data_source: magazine,data_source: academic_literature,data_source: note,data_source: newspaper,mean
MinerU-0.9.3,0.055,0.124,0.033,0.102,0.159,0.072,0.025,0.984,0.148,0.189
Marker-0.2.17,0.188,0.327,0.087,0.292,0.423,0.134,0.102,0.470,0.270,0.255
Marker-1.2.3,0.074,0.340,0.089,0.319,0.452,0.153,0.059,0.651,0.192,0.259
Mathpix,0.131,0.168,0.202,0.199,0.278,0.138,0.091,0.631,0.648,0.276
docling,0.709,0.916,0.994,0.801,0.853,0.720,0.326,1.000,0.996,0.813
GOT-OCR,0.117,0.222,0.067,0.132,0.204,0.198,0.179,0.388,0.771,0.253
Nougat,0.739,0.958,1.000,0.820,0.930,0.830,0.214,0.991,0.871,0.817
GPT4o,0.163,0.163,0.348,0.187,0.281,0.173,0.146,0.607,0.751,0.313
Qwen2-VL-72B,0.096,0.080,0.145,0.148,0.219,0.065,0.315,0.298,0.790,0.240
InternVL2-Llama3-76B,0.216,0.098,0.162,0.184,0.247,0.150,0.419,0.226,0.903,0.289


In [8]:
# page special issue
dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["page"]["Edit_dist"])
df2 = pd.DataFrame(dict_list, index=offical_names)

reordered_df2 = df2.round(3)
reordered_df2

selected_columns = reordered_df2[['fuzzy_scan', 'watermark', 'colorful_backgroud']]
# calculate mean and variance
selected_columns['mean'] = selected_columns.mean(axis=1).round(3)
selected_columns['variance'] = selected_columns.var(axis=1).round(4)

# selected_columns.to_csv('./page_issue.csv')
selected_columns

/mnt/petrelfs/ouyanglinke/.local/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/mnt/petrelfs/ouyanglinke/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,fuzzy_scan,watermark,colorful_backgroud,mean,variance
MinerU-0.9.3,0.150,0.151,0.107,0.136,0.0004
Marker-0.2.17,0.286,0.436,0.290,0.337,0.0049
Marker-1.2.3,0.333,0.484,0.319,0.379,0.0056
Mathpix,0.294,0.290,0.182,0.255,0.0027
docling,0.938,0.807,0.827,0.857,0.0033
GOT-OCR,0.175,0.190,0.186,0.184,0.0000
Nougat,0.934,0.915,0.873,0.907,0.0006
GPT4o,0.263,0.195,0.184,0.214,0.0012
Qwen2-VL-72B,0.101,0.157,0.114,0.124,0.0006
InternVL2-Llama3-76B,0.120,0.197,0.155,0.157,0.0010


In [9]:
# reading order under different layout
dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['reading_order']["page"]["Edit_dist"])

df3 = pd.DataFrame(dict_list, index=offical_names)

reordered_df3 = df3.round(3)


selected_columns3 = reordered_df3[["layout: single_column", "layout: double_column", "layout: three_column", "layout: other_layout"]]
# calculate mean and variance
selected_columns3['mean'] = selected_columns3.mean(axis=1).round(3)
selected_columns3['variance'] = selected_columns3.var(axis=1).round(4)

# selected_columns3.to_csv('./layout.csv')
selected_columns3

/mnt/petrelfs/ouyanglinke/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/mnt/petrelfs/ouyanglinke/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,layout: single_column,layout: double_column,layout: three_column,layout: other_layout,mean,variance
MinerU-0.9.3,0.311,0.101,0.117,0.376,0.226,0.0143
Marker-0.2.17,0.231,0.251,0.309,0.378,0.292,0.0033
Marker-1.2.3,0.299,0.299,0.149,0.363,0.277,0.0062
Mathpix,0.189,0.175,0.225,0.413,0.250,0.0091
docling,0.758,0.535,0.829,0.701,0.706,0.0118
GOT-OCR,0.163,0.145,0.272,0.468,0.262,0.0165
Nougat,0.852,0.601,0.676,0.873,0.750,0.0133
GPT4o,0.109,0.204,0.254,0.426,0.248,0.0132
Qwen2-VL-72B,0.098,0.248,0.514,0.429,0.322,0.0260
InternVL2-Llama3-76B,0.082,0.312,0.682,0.444,0.380,0.0472


In [10]:
# text attribute
dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["group"]["Edit_dist"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4.round(3)

selected_columns = df4[["text_language: text_english", "text_language: text_simplified_chinese", "text_language: text_en_ch_mixed", "text_background: white", "text_background: single_colored", "text_background: multi_colored"]]

# selected_columns.to_csv('.text_attribute.csv')
selected_columns

,text_language: text_english,text_language: text_simplified_chinese,text_language: text_en_ch_mixed,text_background: white,text_background: single_colored,text_background: multi_colored
MinerU-0.9.3,0.124,0.206,0.742,0.163,0.147,0.513
Marker-0.2.17,0.267,0.390,0.499,0.340,0.389,0.497
Marker-1.2.3,0.163,0.379,0.747,0.303,0.396,0.594
Mathpix,0.173,0.774,0.538,0.675,0.554,0.570
docling,0.538,0.980,0.931,0.859,0.798,0.927
GOT-OCR,0.252,0.763,0.266,0.669,0.595,0.440
Nougat,0.588,0.991,0.983,0.875,0.935,0.972
GPT4o,0.171,0.647,0.322,0.536,0.423,0.406
Qwen2-VL-72B,0.337,0.575,0.310,0.537,0.400,0.233
InternVL2-Llama3-76B,0.418,0.606,0.251,0.589,0.366,0.221


In [11]:
# table attribute
dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['table']["group"]["TEDS"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4 * 100
df4 = df4.round(1)

selected_columns = df4[["language: table_en", "language: table_simplified_chinese", "language: table_en_ch_mixed", "line: full_line", "line: less_line", "line: fewer_line", "line: wireless_line", 
                        "with_span: True", "with_span: False", "include_equation: True", "include_equation: False", "include_background: True", "include_background: False", "table_layout: vertical", "table_layout: horizontal"]]

# selected_columns.to_csv('./table_attribute.csv')
selected_columns

,language: table_en,language: table_simplified_chinese,language: table_en_ch_mixed,line: full_line,line: less_line,line: fewer_line,line: wireless_line,with_span: True,with_span: False,include_equation: True,include_equation: False,include_background: True,include_background: False,table_layout: vertical,table_layout: horizontal
MinerU-0.9.3,75.1,59.3,79.1,59.4,71.6,69.7,60.0,63.6,65.3,66.0,64.4,59.2,67.5,3.0,65.8
Marker-0.2.17,52.0,42.0,44.1,41.0,53.8,46.5,51.4,43.3,46.1,42.0,45.8,43.6,45.8,6.3,45.7
Marker-1.2.3,64.9,47.3,49.8,44.5,61.8,59.0,63.6,52.6,52.7,53.2,52.5,48.0,54.9,35.5,52.9
Mathpix,75.4,63.2,71.3,67.4,77.3,66.3,25.5,70.3,65.4,68.7,66.7,59.7,70.8,19.2,67.9
docling,58.2,24.1,30.1,25.9,33.5,48.0,22.7,34.0,34.7,43.8,32.3,28.8,37.3,4.9,35.0
GOT-OCR,51.7,46.2,49.0,45.5,48.3,51.3,46.2,46.0,48.9,45.7,48.4,39.8,51.9,0.0,48.7
Nougat,36.2,0.3,0.0,6.1,3.5,22.1,0.0,15.0,8.9,21.0,8.7,2.6,15.2,0.0,11.2
GPT4o,71.1,58.0,57.3,62.5,68.7,61.3,31.2,56.8,64.7,60.8,62.2,61.4,62.2,14.2,62.7
Qwen2-VL-72B,57.0,62.5,72.1,70.2,63.5,48.1,57.2,49.3,67.6,47.9,64.3,63.1,60.2,41.4,61.5
InternVL2-Llama3-76B,60.9,58.5,65.4,58.8,65.3,58.3,55.6,49.0,65.1,53.3,60.9,58.8,59.8,6.9,60.3


In [5]:
# text OCR

ocr_types = ['OmniDocBench_easyocr_text_ocr', 'OmniDocBench_openocr_text_ocr']
offical_names = ocr_types

result_folder = '/mnt/petrelfs/ouyanglinke/OmniDocBench/result'

dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result["group"]["Edit_dist"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4.round(3)

selected_columns = df4[["text_language: text_english", "text_language: text_simplified_chinese", "text_language: text_en_ch_mixed", "text_background: white", "text_background: single_colored", "text_background: multi_colored", "text_rotate: normal", "text_rotate: rotate90", "text_rotate: rotate270", "text_rotate: horizontal"]]

# selected_columns.to_csv('.text_attribute.csv')
selected_columns

,text_language: text_english,text_language: text_simplified_chinese,text_language: text_en_ch_mixed,text_background: white,text_background: single_colored,text_background: multi_colored,text_rotate: normal,text_rotate: rotate90,text_rotate: rotate270,text_rotate: horizontal
OmniDocBench_easyocr_text_ocr,0.26,0.398,0.445,0.366,0.287,0.388,0.360,0.970,0.997,0.926
OmniDocBench_openocr_text_ocr,0.07,0.068,0.106,0.069,0.058,0.081,0.069,0.038,0.891,0.025
